In [ ]:
import portal
import glob
import joblib
from joblib import Parallel, delayed
import pcmsc

%load_ext autoreload
%autoreload 2

In [ ]:
doi = "F7CR5RW8"
title = "Oceanographic measurements obtained offshore of the Elwha River delta in coordination with the Elwha River Restoration Project, Washington, USA, 2010-2014"
summary = """First Release: Aug 2017
Revised: May 2018 (ver. 1.1)

Time-series data of velocity, pressure, turbidity, conductivity, and temperature were collected near the mouth of the Elwha River, Washington, USA, from December 2010 through October 2014, for the Department of Interior’s Elwha River Restoration project. As part of this project, the U.S. Geological Survey studied the effects of renewed sediment supplies on the coastal ecosystems before, during, and following the removal of two dams, Elwha and Glines Canyon, from the Elwha River. Removal of the dams reintroduced sediment stored in the reservoirs to the river, and the river moved much of this sediment to the coast. 
 
Several benthic tripods were instrumented with oceanographic sensors to collect the time-series data. Initial deployment in December 2010 consisted of one tripod about 1 km east of the Elwha River mouth (Tripod A). In March of 2011, an identical tripod (Tripod B) was placed about 1 km west of the river mouth. A mooring was added to the western site in July 2012 to measure turbidity and conductivity near the surface. A third tripod was placed in deeper water (50 m) directly offshore of the river mouth in an attempt to characterize sediment gravity flows near the seafloor if they occurred (Tripod C). Exceptional sedimentation was observed near the original tripod site A during the winter of 2013-2014. As a result, the tripod was relocated further east in April 2013 and renamed Tripod D.

Please check metadata and instrument information carefully for applicable time periods of specific data, as individual instrument deployment times and duration of the time series vary.

The naming convention for the NetCDF files included in this release is a 12-character alphanumeric code (ELWYYJKLNNXX.nc) where:

ELW is a 3-digit alphabetic-code for this experiment located at the mouth of the Elwha River
YY is the 2 digit year at the time of deployment
J is the location with respect to the river mouth [A, East (December 2010 to April 2013); B, West; C, Offshore; D, East (April 2013 to March 2014)]
K is the deployment number (1-9; beginning and ending dates of each deployment are given below)
L is the instrument package type (T, tripod; M, surface mooring)
NN indicates the position of instrument on the surface mooring (01, nearest the surface; NN increases with depth)
XX denotes the instrument or data type (wh, RDInstruments ADCP current data; wv, RDInstruments ADCP derived wave parameters; nx, Falmouth Scientific NXIC CTD; aq, Aquatec Aqualogger OBS; bl, RBR, Ltd CTD; sc, SeaBird Electonics SBE16+ CT)

Some derived parameters are included in these data.

Deployment dates:
1. Dec 2010 to Mar 2011
2. Mar 2011 to Sep 2011
3. Sep 2011 to Mar 2012
4. Mar 2012 to Aug 2012
5. Aug 2012 to Jan 2013
6. Jan 2013 to Jun 2013
7. Jun 2013 to Dec 2013
8. Dec 2013 to Mar 2014
9. Mar 2014 to Oct 2014"""

In [ ]:
%%time
files = glob.glob('../pcmsc/doi-' + doi + '/*.nc')
for f in files:
    print(f)
    c = pcmsc.convert(f, doi, title, summary)
    print(c)

In [ ]:
files = glob.glob('../pcmsc/doi-' + doi + '/clean/*wh*.nc')
for f in files:
    print(f)
    cc = portal.check_compliance_system(f, "cf:1.6")
    print(cc.stdout.decode('utf-8'))
    cc = portal.check_compliance_system(f, "acdd")
    print(cc.stdout.decode('utf-8'))

In [ ]:
import xarray as xr
ds = xr.open_dataset('../pcmsc/doi-F7CR5RW8/clean/ELW13D7T03aq.nc')
print(ds.attrs['title'])
# ds.Trb_980.plot()
ds.close()

In [ ]:
%%time
files = glob.glob('../pcmsc/doi-' + doi + '/*.nc')
# need to specify multiprocessing so we get print statements
Parallel(n_jobs=8, backend='multiprocessing')(delayed(convert)(f) for f in files)